# Limpieza de Datos Dataset Promociones-20190715

In [1]:
import pandas as pd
from utils_functions import convert_file, upload_to_bucket

#### Importar el dataset y observar los primeros registros con head y verificar los tipos de datos con el método info.

In [2]:
promo_col_names = [
        'idb', 'id_producto', 'fecha_desde', 'fecha_hasta', 'ventas_estimadas_und',
        'nro_prom', 'fecha_trans', 'porcentaje_dcto'
]

In [3]:
promo_df = pd.read_csv("../data/VMI_Promociones-20190715.txt",
                       encoding="utf-8",
                       header=None,
                       delimiter='\t',
                       names=promo_col_names)

In [4]:
promo_df.head()

,idb,id_producto,fecha_desde,fecha_hasta,ventas_estimadas_und,nro_prom,fecha_trans,porcentaje_dcto
0,242,228217,20161023,20161030,0.98,436830,20161005,7.0
1,242,228217,20161023,20161030,0.98,436830,20161006,7.0
2,242,228217,20161023,20161030,0.98,436830,20161007,7.0
3,242,228217,20161023,20161030,0.98,436830,20161008,7.0
4,242,228217,20161023,20161030,0.98,436830,20161009,7.0


In [5]:
promo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285253 entries, 0 to 285252
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   idb                   285253 non-null  int64  
 1   id_producto           285253 non-null  int64  
 2   fecha_desde           285253 non-null  int64  
 3   fecha_hasta           285253 non-null  int64  
 4   ventas_estimadas_und  285253 non-null  float64
 5   nro_prom              285253 non-null  int64  
 6   fecha_trans           285253 non-null  int64  
 7   porcentaje_dcto       285253 non-null  float64
dtypes: float64(2), int64(6)
memory usage: 17.4 MB


#### Ajustes al dataframe
- Convertir la columnas fecha_desde, fecha_hasta y fecha_trans a tipo de dato datetime.
- Verificar si existen valores nulos y/o repetidos.
- Obtener las fechas máximas y minimas de transaciones.
- Crear una nueva columnas llamada duracion_promo para obtener el delta de tiempo entre la fecha_desde y la fecha_hasta de una promoción.
- Obtener una análisis descriptivo de los datos.

In [6]:
promo_df = convert_file(promo_df, 'fecha_desde',  'date')

In [7]:
promo_df = convert_file(promo_df, 'fecha_hasta',  'date')

In [8]:
promo_df = convert_file(promo_df, 'fecha_trans', 'date')

In [9]:
promo_df.isnull().sum(axis=0)

idb                     0
id_producto             0
fecha_desde             0
fecha_hasta             0
ventas_estimadas_und    0
nro_prom                0
fecha_trans             0
porcentaje_dcto         0
dtype: int64

In [10]:
promo_df.duplicated().sum()

0

In [11]:
min_date = promo_df['fecha_desde'].min()
max_date = promo_df['fecha_hasta'].max()


print(f"Minimum date: {min_date}")
print(f"Maximum date: {max_date}")

Minimum date: 2016-07-19 00:00:00
Maximum date: 2019-07-16 00:00:00


In [12]:
min_date = promo_df['fecha_trans'].min()
max_date = promo_df['fecha_trans'].max()

print(f"Minimum date: {min_date}")
print(f"Maximum date: {max_date}")

Minimum date: 2016-07-14 00:00:00
Maximum date: 2019-07-14 00:00:00


In [13]:
# Crear una nueva columna llamada duracion de la promoción.
promo_df['duracion_promo_delta'] = promo_df['fecha_hasta'] - promo_df['fecha_desde']

# Convertir timedelta to float
promo_df['duracion_promo'] = promo_df['duracion_promo_delta'].dt.days.astype(float)

# Drop la columna duracion_promo
promo_df.drop(columns=['duracion_promo_delta'], inplace=True)

In [14]:
promo_df.describe()

,idb,id_producto,ventas_estimadas_und,nro_prom,porcentaje_dcto,duracion_promo
count,285253.000000,285253.000000,285253.000000,285253.000000,285253.000000,285253.000000
mean,7065.578963,388393.367197,24.622701,471275.031435,8.394103,8.394103
std,3169.254446,83906.420820,30.314661,37596.939924,2.893570,2.893570
min,242.000000,164146.000000,0.000000,425196.000000,3.000000,3.000000
25%,5207.000000,342682.000000,6.000000,436830.000000,7.000000,7.000000
50%,9026.000000,427445.000000,12.520000,456807.000000,7.000000,7.000000
75%,9159.000000,458395.000000,30.440000,502702.000000,8.000000,8.000000
max,9977.000000,507528.000000,300.820000,541066.000000,18.000000,18.000000


In [15]:
promo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285253 entries, 0 to 285252
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   idb                   285253 non-null  int64         
 1   id_producto           285253 non-null  int64         
 2   fecha_desde           285253 non-null  datetime64[ns]
 3   fecha_hasta           285253 non-null  datetime64[ns]
 4   ventas_estimadas_und  285253 non-null  float64       
 5   nro_prom              285253 non-null  int64         
 6   fecha_trans           285253 non-null  datetime64[ns]
 7   porcentaje_dcto       285253 non-null  float64       
 8   duracion_promo        285253 non-null  float64       
dtypes: datetime64[ns](3), float64(3), int64(3)
memory usage: 19.6 MB


In [16]:
promo_df.head()

,idb,id_producto,fecha_desde,fecha_hasta,ventas_estimadas_und,nro_prom,fecha_trans,porcentaje_dcto,duracion_promo
0,242,228217,2016-10-23,2016-10-30,0.98,436830,2016-10-05,7.0,7.0
1,242,228217,2016-10-23,2016-10-30,0.98,436830,2016-10-06,7.0,7.0
2,242,228217,2016-10-23,2016-10-30,0.98,436830,2016-10-07,7.0,7.0
3,242,228217,2016-10-23,2016-10-30,0.98,436830,2016-10-08,7.0,7.0
4,242,228217,2016-10-23,2016-10-30,0.98,436830,2016-10-09,7.0,7.0


#### Observaciones dataset Promociones-20190715

1. El dataset contiene la información referente a las promociones realizadas desde 2016-07-19 hasta el 2019-07-16.
2. El dataset contiene 285253 registros y 9 columnas descritas así:
    - idb:	Identificador interno de Boca
    - id_producto:	Identificador interno de Artículo
    - fecha-desde: 	Fecha desde de la vigencia de la Promoción
    - fecha_hasta:	Fecha hasta de la vigencia de la Promoción
    - ventas_estimadas_und: Estimado de ventas en la promoción en Unidades
    - nro_prom: Número de la Promoción
    - fecha_trans: Fecha de la Información
    - porcentaje_dcto: Porcentaje de Descuento
    - duracion_promo: La diferencia entre fecha_desde y fecha_hasta.
3. No se observan valores NAN ó duplicados.
4. La media de ventas estimadas es de 24.62 unidades.
5. El máximo porcentaje de descuento es de 18% y el mínimo es de 3%.
6. La duración máxima de una promoción es de 18 días.
7. La duración media de una promoción es de 8 días.

### Guardar el dataset limpio

In [17]:
promo_df.to_csv('../clean_data/promociones_clean.csv', index=False)

#### Subir csv a google cloud storage

In [18]:
upload_to_bucket('deodorants', '../clean_data/promociones_clean.csv', 'promociones')

File ../clean_data/promociones_clean.csv uploaded successfully to bucket deodorants/promociones
